In [ ]:
import os
import json
import gpxpy
import folium
import geopandas as gpd
from shapely.geometry import LineString
from geopy.distance import geodesic
import tkinter as tk
from tkinter import messagebox
import webbrowser
import time

# Paths
gpx_folder = "/Users/chris/Desktop/Dektop/health/apple_health_export/workout-routes_updated"
cache_file = "processed_gpx_files.json"
map_file = "map_of_routes_temp.html"

# Threshold distance in meters to identify "unreasonable" jumps
def clear_cache():
    """Clear the processed files cache."""
    if os.path.exists(cache_file):
        os.remove(cache_file)
        print("Cache cleared. All files will be reprocessed.")
    else:
        print("No cache file found. Nothing to clear.")


DISTANCE_THRESHOLD_METERS = 200

def load_gpx_track(file_path):
    """Load the track points from a GPX file and return filtered LineStrings to exclude large jumps."""
    print(f"Processing file: {file_path}")
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as gpx_file:
        gpx = gpxpy.parse(gpx_file)
        segments = []

        for track in gpx.tracks:
            for segment in track.segments:
                points = []
                for i in range(1, len(segment.points)):
                    prev_point = segment.points[i - 1]
                    current_point = segment.points[i]
                    distance = geodesic((prev_point.latitude, prev_point.longitude), 
                                        (current_point.latitude, current_point.longitude)).meters

                    if distance <= DISTANCE_THRESHOLD_METERS:
                        points.append((current_point.longitude, current_point.latitude))
                    else:
                        if len(points) > 1:
                            segments.append(LineString(points))
                        points = [(current_point.longitude, current_point.latitude)]

                if len(points) > 1:
                    segments.append(LineString(points))

    print(f"Finished processing file: {file_path}, segments found: {len(segments)}")
    return segments

def load_cache():
    """Load the list of processed GPX files from the cache."""
    if os.path.exists(cache_file):
        with open(cache_file, 'r') as f:
            return set(json.load(f))
    return set()

def save_cache(processed_files):
    """Save the list of processed GPX files to the cache."""
    with open(cache_file, 'w') as f:
        json.dump(list(processed_files), f)

def create_or_update_map(gpx_folder):
    """Create or update a map with GPX tracks overlayed."""
    print("Loading cache...")
    processed_files = load_cache()
    print(f"Cache loaded. {len(processed_files)} files already processed.")

    # Load existing map or create a new one
    if os.path.exists(map_file):
        print("Map file exists. Updating map...")
        m = folium.Map(location=[0, 0], zoom_start=2, tiles='cartodb positron')
    else:
        print("Map file does not exist. Creating a new map...")
        m = folium.Map(location=[0, 0], zoom_start=2, tiles='cartodb positron')

    for filename in os.listdir(gpx_folder):
        if filename.endswith('.gpx') and filename not in processed_files:
            print(f"New file detected: {filename}")
            file_path = os.path.join(gpx_folder, filename)
            track_segments = load_gpx_track(file_path)
            for segment in track_segments:
                gdf = gpd.GeoDataFrame(geometry=[segment], crs="EPSG:4326")
                for line in gdf.geometry:
                    folium.PolyLine(locations=[(lat, lon) for lon, lat in line.coords],
                                    color="blue", weight=2, opacity=0.6).add_to(m)
            processed_files.add(filename)

    print("Saving updated map...")
    save_cache(processed_files)
    m.save(map_file)
    print(f"Map saved as {map_file}")

    return m

def display_map():
    clear_cache()
    """Generate the map and show it in the browser."""
    print("Generating map...")
    create_or_update_map(gpx_folder)
    print("Map generation complete. Opening in browser...")
    time.sleep(0.5)  # Small delay for Safari to access the file
    webbrowser.get('safari').open(f"file://{os.path.abspath(map_file)}")

# GUI setup
app = tk.Tk()
app.title("GPX Map Overlay Application")
app.geometry("400x200")

label = tk.Label(app, text="Generate Map from GPX Tracks", font=("Helvetica", 14))
label.pack(pady=20)

button = tk.Button(app, text="Generate Map", command=display_map)
button.pack(pady=10)

app.mainloop()

Cache cleared. All files will be reprocessed.
Generating map...
Loading cache...
Cache loaded. 0 files already processed.
Map file exists. Updating map...
New file detected: Outdoor Walk-Route-20240704_104506.gpx
Processing file: /Users/chris/Desktop/Dektop/health/apple_health_export/workout-routes_updated/Outdoor Walk-Route-20240704_104506.gpx
Finished processing file: /Users/chris/Desktop/Dektop/health/apple_health_export/workout-routes_updated/Outdoor Walk-Route-20240704_104506.gpx, segments found: 0
New file detected: Outdoor Run-Route-20240201_120116.gpx
Processing file: /Users/chris/Desktop/Dektop/health/apple_health_export/workout-routes_updated/Outdoor Run-Route-20240201_120116.gpx
Finished processing file: /Users/chris/Desktop/Dektop/health/apple_health_export/workout-routes_updated/Outdoor Run-Route-20240201_120116.gpx, segments found: 1
New file detected: Outdoor Walk-Route-20240905_064540.gpx
Processing file: /Users/chris/Desktop/Dektop/health/apple_health_export/workout-ro

In [1]:
!pip install gpxpy folium geopandas shapely

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 5.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 5.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 5.4 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
!pip install geopy


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
import os
import gpxpy
import folium
import geopandas as gpd
from shapely.geometry import LineString
import tkinter as tk
from tkinter import messagebox
import webbrowser
import time
from geopy.distance import geodesic
import json

# Paths
gpx_folder = "/Users/chris/Desktop/Dektop/health/apple_health_export/workout-routes_updated"
processed_files_record = "processed_files.json"
DISTANCE_THRESHOLD_METERS = 200  # Threshold distance in meters to identify "unreasonable" jumps

# Load or initialize processed files record
if os.path.exists(processed_files_record):
    with open(processed_files_record, 'r') as f:
        processed_files = json.load(f)
else:
    processed_files = []

def load_gpx_track(file_path):
    """Load track points from a GPX file and return filtered LineStrings to exclude large jumps."""
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as gpx_file:
        gpx = gpxpy.parse(gpx_file)
        segments = []

        for track in gpx.tracks:
            for segment in track.segments:
                points = []
                for i in range(1, len(segment.points)):
                    prev_point = segment.points[i - 1]
                    current_point = segment.points[i]
                    distance = geodesic((prev_point.latitude, prev_point.longitude), 
                                        (current_point.latitude, current_point.longitude)).meters
                    
                    # Check if the distance is within the threshold
                    if distance <= DISTANCE_THRESHOLD_METERS:
                        points.append((current_point.longitude, current_point.latitude))
                    else:
                        # Save the current segment and start a new one for large jumps
                        if len(points) > 1:
                            segments.append(LineString(points))
                        points = [(current_point.longitude, current_point.latitude)]  # Start new segment

                # Add the last segment if it has enough points
                if len(points) > 1:
                    segments.append(LineString(points))
    
    return segments

def create_map(gpx_folder):
    """Create a map with GPX tracks overlayed and display it in a browser."""
    m = folium.Map(location=[0, 0], zoom_start=2, tiles='cartodb positron')

    # Iterate through all GPX files, skipping already processed ones
    new_processed_files = []
    for filename in os.listdir(gpx_folder):
        if filename.endswith('.gpx') and filename not in processed_files:
            file_path = os.path.join(gpx_folder, filename)
            track_segments = load_gpx_track(file_path)
            for segment in track_segments:
                gdf = gpd.GeoDataFrame(geometry=[segment], crs="EPSG:4326")
                for line in gdf.geometry:
                    folium.PolyLine(locations=[(lat, lon) for lon, lat in line.coords],
                                    color="blue", weight=2, opacity=0.6).add_to(m)
            new_processed_files.append(filename)
    
    # Update the processed files record
    processed_files.extend(new_processed_files)
    with open(processed_files_record, 'w') as f:
        json.dump(processed_files, f)

    # Save to a temporary file and open in the browser
    temp_path = "map_of_routes_temp.html"
    m.save(temp_path)

    # Wait a moment for the file to ensure Safari can access it
    time.sleep(0.5)
    webbrowser.get('safari').open(f"file://{os.path.abspath(temp_path)}")

def generate_map():
    """Generate the map and show a completion message."""
    create_map(gpx_folder)
    messagebox.showinfo("Map Generated", "Map displayed in your browser.")

# Setting up the GUI
app = tk.Tk()
app.title("GPX Map Overlay Application")
app.geometry("400x200")

# Adding button and label
label = tk.Label(app, text="Generate Map from GPX Tracks", font=("Helvetica", 14))
label.pack(pady=20)

button = tk.Button(app, text="Generate Map", command=generate_map)
button.pack(pady=10)

# Run the app
app.mainloop()